<a href="https://colab.research.google.com/github/ks-chauhan/Kaggle-challenges/blob/main/kaggle_titanic_survival_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import torch
from torch import nn
from tqdm.auto import tqdm

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("Titanic_Survival_Train.csv")

In [ ]:
df.tail(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [ ]:
df[(df["Survived"]==1) & (df['Parch']==0)].shape[0]

233

In [ ]:
features=["Pclass","Sex","Age","SibSp","Parch","Fare","Embarked"]
target="Survived"

In [ ]:
df=df[features+[target]].dropna()

In [ ]:
df['Sex']=df['Sex'].map({'male':0,'female':1})

In [ ]:
df['Embarked']=df['Embarked'].map({'C':0,'Q':1,'S':2})

In [ ]:
df["Age"] = df["Age"].apply(lambda age: 0 if age <= 12 else
                                       1 if age <= 19 else
                                       2 if age <= 59 else 3)

In [ ]:
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3,0,2,1,0,7.2500,2,0
1,1,1,2,1,0,71.2833,0,1
2,3,1,2,0,0,7.9250,2,1
3,1,1,2,1,0,53.1000,2,1
4,3,0,2,0,0,8.0500,2,0


In [ ]:
X_Train_array=df[features].values
y_Train_array=df[target].values

In [ ]:
X_Train=torch.tensor(X_Train_array,dtype=torch.float32)
y_Train=torch.tensor(y_Train_array,dtype=torch.float32)

In [ ]:
X_Train[:4]

tensor([[ 3.0000,  0.0000,  2.0000,  1.0000,  0.0000,  7.2500,  2.0000],
        [ 1.0000,  1.0000,  2.0000,  1.0000,  0.0000, 71.2833,  0.0000],
        [ 3.0000,  1.0000,  2.0000,  0.0000,  0.0000,  7.9250,  2.0000],
        [ 1.0000,  1.0000,  2.0000,  1.0000,  0.0000, 53.1000,  2.0000]])

In [ ]:
class TitanicSurvivalModelV0(nn.Module):
  def __init__(self,
               input_size:int,
               hidden_size:int,
               output_size:int):
    super().__init__()
    self.Layer1=nn.Sequential(
        nn.Linear(in_features=input_size,out_features=hidden_size),
        nn.ReLU(),
        nn.Linear(in_features=hidden_size,out_features=hidden_size),
        nn.ReLU(),
        nn.Linear(in_features=hidden_size,out_features=output_size)
    )
  def forward(self,X:torch.Tensor):
    X=self.Layer1(X)
    return X

In [ ]:
model_0=TitanicSurvivalModelV0(input_size=7,hidden_size=14,output_size=1)

In [ ]:
loss_fn=torch.nn.BCEWithLogitsLoss()
optimizer=torch.optim.Adam(params=model_0.parameters(),
                          lr=0.005)

In [ ]:
pred=model_0(X_Train)
loss=loss_fn(pred.squeeze(),y_Train)
print(loss)

tensor(0.2462, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


In [ ]:
epochs=50000
for epoch in tqdm(range(epochs)):
  model_0.train()
  pred=model_0(X_Train)
  loss=loss_fn(pred.squeeze(),y_Train)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if(epoch%400 == 0):
    print(f"current loss : {loss}")

  0%|          | 0/50000 [00:00<?, ?it/s]

current loss : 0.24616438150405884
current loss : 0.2455962896347046
current loss : 0.2454516440629959
current loss : 0.24521788954734802
current loss : 0.24544070661067963
current loss : 0.24494054913520813
current loss : 0.24488581717014313
current loss : 0.24505898356437683
current loss : 0.2477739155292511
current loss : 0.24457134306430817
current loss : 0.24443954229354858
current loss : 0.2500993609428406
current loss : 0.24413803219795227
current loss : 0.2443877011537552
current loss : 0.2440253496170044
current loss : 0.24399416148662567
current loss : 0.243833988904953
current loss : 0.24374087154865265
current loss : 0.24369913339614868
current loss : 0.24362070858478546
current loss : 0.24359336495399475
current loss : 0.24370236694812775
current loss : 0.243446484208107
current loss : 0.2433089166879654
current loss : 0.2436731457710266
current loss : 0.2434217929840088
current loss : 0.24319013953208923
current loss : 0.2431115359067917
current loss : 0.24401888251304626

In [ ]:
y_Train.shape

torch.Size([712])